In [1]:
import pandas as pd
import numpy as np
import fastai
from fastai.vision.all import *

import dill

In [2]:
fastai.__version__, torch.__version__

('2.3.0', '1.7.0')

In [3]:
models = [
    '../input/fgvc8hotel/export_res50_HQAdam.pkl', #v3
    '../input/fgvc8hotel/export_dn161_Fa_CE_bs32.pkl', #v7
    '../input/fgvc8hotel/export_dn161_Fa_FL_bs32.pkl', #v8
    '../input/fgvc8hotel/export_res101_Fall_HQAdam.pkl', #v5
    '../input/fgvc8hotel/export_res101_Fall_5it_4.pkl', #v11
    '../input/hotel-train-fastai-densnet161/export_dn161_kaggle_notebook.pkl' # kaggle v2
]

In [4]:
image_path = '../input/hotel-id-2021-fgvc8/test_images/' 

In [5]:
submission = pd.read_csv('../input/hotel-id-2021-fgvc8/sample_submission.csv')

test = submission.copy()

test['image'] = '../input/hotel-id-2021-fgvc8/test_images/' +  test['image']
#test['image'] = 'hotel-id-2021-fgvc8/test_images/' + test['image']
test

,image,hotel_id
0,../input/hotel-id-2021-fgvc8/test_images/99e91ad5f2870678.jpg,36363 53586 18807 64314 60181
1,../input/hotel-id-2021-fgvc8/test_images/b5cc62ab665591a9.jpg,36363 53586 18807 64314 60181
2,../input/hotel-id-2021-fgvc8/test_images/d5664a972d5a644b.jpg,36363 53586 18807 64314 60181


In [6]:
probs = None
for model in models:
    learn = load_learner(fname = Path(model), cpu=False, pickle_module=dill)
    test_dl = learn.dls.test_dl(test)
    probs_temp, _ = learn.tta(dl=test_dl, n=4)
    
    if probs == None:
        probs = probs_temp
    else:
        probs += probs_temp


In [7]:
preds_idx = probs.topk(5)[1]

In [8]:
preds = [' '.join(map(str, learn.dls.vocab[pred])) for pred in preds_idx]
preds[:5]

['971 34608 9874 63210 32842',
 '62229 42885 41583 52660 44842',
 '60667 30644 2829 3011 12475']

In [9]:
submission['hotel_id'] = preds

submission.to_csv('submission.csv', index = False)

submission.head()

,image,hotel_id
0,99e91ad5f2870678.jpg,971 34608 9874 63210 32842
1,b5cc62ab665591a9.jpg,62229 42885 41583 52660 44842
2,d5664a972d5a644b.jpg,60667 30644 2829 3011 12475
